In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.gridspec import GridSpec
from ipywidgets import interact_manual

%matplotlib inline

In [2]:
PREFILTER = True
PREFILTER_THRESHOLD = 0.2 # Data excluded if Brand variance greather than 0.2M or less than -0.2M

WINDOW_SIZE = 13 # Timeseries window size in months

In [3]:
from helper_save_load import load_from_pickle

df_c = load_from_pickle("data/Labeled_comments2.pkl")

In [4]:
# These brands are in df_c but do not appear in the variance dataframe df_v
ignored_brands = ["IGNORE",
"57-CAMAY BAR SOAP Brand",
"71-LBP SH/CD Roll-up Brand",
"72-LBP SHOWER Roll-up Brand"]

df_c = df_c[~df_c["Brand_1"].isin(ignored_brands)]

In [5]:
df_c.insert(16, "NoComm", df_c["Commentaries"].isnull())
df_c.insert(16, "Other", (df_c[['Promo', 'Phasing', 'SP&D', 'POS']].sum(axis=1) == 0) & ~df_c['NoComm'])
df_c["Month"] = pd.to_datetime(df_c["Month"]).dt.strftime("%b_%Y")

In [6]:
label_cols = ['Promo', 'Phasing', 'SP&D', 'POS', 'Other', 'NoComm']

territory_label_cols = df_c.columns[18:31].to_list()
territory_labels = df_c[territory_label_cols]

In [7]:
df_a, df_f, df_v = load_from_pickle("data/dataframes_Dollars.pickle")
del df_a, df_f

In [8]:
#territories = [territory for territory, values in df_v.groupby(['Territory']).groups.items()]

In [9]:
if PREFILTER:
    df_c = df_c[(df_c["Variance"] >= PREFILTER_THRESHOLD) | (df_c["Variance"] <= -PREFILTER_THRESHOLD)]

In [10]:
CLASS_PRIORITY = ["Promo", "SP&D", "Phasing", "POS"] # Most important class to predict to least

print('Number of multiclass labels before:', sum(df_c[label_cols].sum(axis=1) > 1))
      
for idx, row in df_c[df_c[label_cols[:4]].sum(axis=1) > 1][label_cols[:4]].iterrows():
    positive_label_idx = min([CLASS_PRIORITY.index(c) for c in row[row > 0].index])
    negative_labels = CLASS_PRIORITY[:]
    negative_labels.pop(positive_label_idx) # List of columns to set to zero
    df_c.loc[idx, negative_labels] = 0

print('Number of multiclass labels after:', sum(df_c[label_cols].sum(axis=1) > 1))

Number of multiclass labels before: 38
Number of multiclass labels after: 0


In [11]:
terr = []
cat = []
for i, row in df_c.iterrows():
    
    t_idx = np.flatnonzero(row.fillna(0)[territory_label_cols])
    c_idx = np.flatnonzero(row.fillna(0)[label_cols])
   
    if t_idx.size > 0:
        terr.append(territory_label_cols[t_idx[0]])
    else:
        terr.append(None)
        
    if c_idx.size > 0:
        cat.append(label_cols[c_idx[0]])
    else:
        cat.append(None)

df_c["Territory"] = terr
df_c["Class"] = cat

In [12]:
TR_BR_grouped = df_v.groupby(["Territory", "Brand"]).sum()

In [ ]:
list(df_v.Territory.unique())

In [15]:
variance_dict = {}
for territory, subterritories in territory_dict.items():
    summed_variance = pd.DataFrame()
    for subterritory in subterritories:
        summed_variance = summed_variance.add(TR_BR_grouped.loc[subterritory], fill_value=0)
        
    variance_dict[territory] = summed_variance

In [16]:
# Brand x territory x months
# Row of zeros if empty
brands = df_v["Brand"].unique()
months = df_v.columns[8:]

all_brands_variance = []

for brand in brands:
    temp = []
    for territory in territory_label_cols:
        if brand in variance_dict[territory].index:
            temp.append(variance_dict[territory].loc[brand].values)
        else:
            temp.append(np.zeros(len(months)))
    all_brands_variance.append(temp)
    
all_brands_variance = np.array(all_brands_variance)
all_brands_variance.shape

(136, 13, 34)

In [17]:
# Generates full list of 12 month timeseries windows and corresponding class labels
# for every *unique* brand-month pair in commentary spreadsheet

labels = []
BR_TR_timeseries_windows = []
previous_commentary_num = -1
brand_month_labels = None

for idx, row in df_c.iterrows():
    brand_idx = np.where(brands==row["Brand_1"])[0][0]
    month_idx = np.where(months==row["Month"])[0][0]
    t_idx = np.flatnonzero(row.fillna(0)[territory_label_cols]) # Index of territory_label for this row
    
    # Due to splitting of commentaries we have repeated brand-month pairs in data
    # Keeping track of which commentary so we don't repeat its timeseries
    current_commentary_num = row["Num"] 
    
    if current_commentary_num != previous_commentary_num:
        # First instance of brand-month pair
        labels.append(brand_month_labels)
        
        brand_month_labels = np.zeros((len(territory_label_cols), len(label_cols)), dtype=int)
        brand_month_labels[:, 5] = 1 # Default NoComm
        
        if t_idx.size > 0:
            brand_month_labels[t_idx[0]] = row[label_cols].fillna(0).values
        
        for i in range(len(territory_label_cols)):

            # Appending WINDOW_SIZE length month windows per brand per territory as input
            if month_idx - WINDOW_SIZE < 0:
                padding = np.zeros(WINDOW_SIZE - month_idx-1) # pad with zeros if less than WINDOW_SIZE
                window = np.concatenate((padding, all_brands_variance[brand_idx, i, :month_idx+1]))
                
            else:
                window = all_brands_variance[brand_idx, i, (month_idx-WINDOW_SIZE+1):month_idx+1]
            BR_TR_timeseries_windows.append(window)
    else:
        # Repeated instances of brand-month pair
        if t_idx.size > 0:
            brand_month_labels[t_idx[0]] = row[label_cols].fillna(0).values
    
    previous_commentary_num = current_commentary_num
    
labels.append(brand_month_labels) # Append the final labels
BR_TR_timeseries_windows = np.array(BR_TR_timeseries_windows)
BR_TR_labels = np.vstack(labels[1:])
BR_TR_timeseries_windows /= 1e6 

print("X data shape:", BR_TR_timeseries_windows.shape) # 286 brand-month pairs * 13 territories, 12 months
print("y data shape:", BR_TR_labels.shape)             # 286 brand-month pairs * 13 territories, 5 labels

X data shape: (3718, 13)
y data shape: (3718, 6)


In [18]:
def plot_brand_month(brand, month):
    
    brand_month = df_c[(df_c["Brand_1"]==brand) & (df_c["Month"]==month)]["Num"]
    
    if brand_month.empty:
        print("*No data available*")
        
    else:
        num = brand_month.iloc[0]
        df_c[(df_c["Brand_1"]==brand) & (df_c["Month"]==month)]["Num"].iloc[0]
    
        num_indices = df_c["Num"].unique()
        num_idx = np.where(num_indices==num)[0][0]

        data = BR_TR_timeseries_windows[num_idx*13:num_idx*13+13]
        labels = np.argmax(BR_TR_labels[num_idx*13:num_idx*13+13], axis=1)
        commentaries = df_c[df_c["Num"] == num][["Brand_1", "Month", "Commentaries", "Territory", "Class"]]
        ylim = (np.min(data)-0.1, np.max(data)+0.1)
        
        
        xmonths = pd.date_range(end=pd.to_datetime(month, format='%b_%Y'), 
                                periods=13, freq='MS').strftime('%b_%y').to_list()
        
        colour_dict = {0:'r', 1:'g', 2:'m', 3:'c', 4:'grey', 5:'b'}
        
        gs00 = GridSpec(5, 3, hspace=0.7)

        fig = plt.figure(figsize=(30,20))
        axs = []

        ax = fig.add_subplot(gs00[0,1])
        axs.append(ax)
        for i in range(1, 5):
            for j in range(3):
                ax = fig.add_subplot(gs00[i,j])
                axs.append(ax)
                
        for i, ax in enumerate(axs):
            ax.hlines(0, -1, WINDOW_SIZE, linewidth=0.5)
            ax.plot(data[i], colour_dict[labels[i]], linewidth=3)
            ax.set_xlim(-1, WINDOW_SIZE)
            ax.set_ylim(ylim)
            ax.tick_params(labelsize=16)
            ax.set_xticks(np.arange(0, WINDOW_SIZE))
            ax.set_xticklabels(xmonths, rotation=45)
            ax.set_title(territory_label_cols[i], fontsize=22)
        
        fig.suptitle("Data for ({}, {})".format(brand, month), fontsize=30, y=0.95)
        markers = [plt.Line2D([0,0],[0,0],color=c, marker='o', linestyle='') for c in colour_dict.values()] 
        fig.legend(markers, label_cols, numpoints=1, markerscale=6, 
                   prop={'size': 30}, title='Commentary Type', title_fontsize=32)
        plt.show()
        
        with pd.option_context('display.max_colwidth', -1):
             display(commentaries)

In [19]:
brands = df_c["Brand_1"].unique()
months = df_c["Month"].unique()

In [20]:
from helper_save_load import save_to_pickle
save_to_pickle('external/plot_data.pkl', [df_c, BR_TR_timeseries_windows, BR_TR_labels])
save_to_pickle('external/brands_and_months.pkl', [brands, months])

In [21]:
interact_manual(plot_brand_month, brand=brands, month=months)

interactive(children=(Dropdown(description='brand', options=('KJ-DOVE BODY WASH Brand', '18-DOVE BW MENS+CARE …

<function __main__.plot_brand_month(brand, month)>

In [22]:
df_c.columns[12:17].tolist()

['Promo', 'Phasing', 'SP&D', 'POS', 'Other']

In [ ]:
TR_BR_grouped.loc['M5100 - METRO ONT IC'].index

In [ ]:
TR_BR_grouped.loc['M5101 - METRO QUE IC'].index

In [25]:
import re
search = []    
for comm in df_c['Commentaries']:
    
    if isinstance(comm, str):
        s = re.search('[+-]? *\$?[0-9\.]+[MK]', comm)
        if s is not None:
            search.append(s.group())
        else:
            search.append('')
    else:
        search.append('')

# df['col1'] = search

In [26]:
search = [s.replace('$', '').replace(' ', '') for s in search]

comm_variances = []

for s in search:

    if s == '':
        comm_variances.append(np.nan)
    elif 'K' in s:
        comm_variances.append(float(s[:-1])/1000)
    elif 'M' in s:
        comm_variances.append(float(s[:-1]))

In [27]:
df_c['CommVar'] = comm_variances

In [28]:
data_idx = []
for i, row in df_c[['Num', 'Territory']].iterrows():
    t = row['Territory']
    if t is not None:
        data_idx.append((row['Num'], territory_label_cols.index(t)))
    else:
        data_idx.append(None)

terr_vars = []        
num_indices = df_c["Num"].unique()      
for x in data_idx:
    if x is None:
        terr_vars.append(np.nan)
    else:
        (num, t_idx) = x
#         print(num)
        num_idx = np.where(num_indices==num)[0][0] 
        terr_vars.append(BR_TR_timeseries_windows[num_idx*13 + t_idx, -1])
    

In [29]:
df_c['TerrVar'] = terr_vars
df_c['VarAbsDiff'] = abs(df_c['CommVar'] - df_c['TerrVar'])

In [30]:
df2 = df_c[df_c['VarAbsDiff'] > 0.5][['Month', 'Brand_1','Commentaries', 'TerrVar', 'CommVar', 'VarAbsDiff', 'Territory']]

In [31]:
len(df2)/len(df_c)

0.056818181818181816

In [ ]:
df_c[['Month','Brand_1','Commentaries', 'CommVar', 'TerrVar', 'Territory']].tail(50)

In [ ]:
df2

In [164]:
df2.to_csv('var_difference.csv', sep='\t', encoding='utf-8')